In [31]:
import numpy as np
from datetime import datetime

In [32]:
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [48]:
class LogicGate:
    
    def __init__(self, gate_name, xdata, tdata, i_node, h1_node, o_node, learning_rate, it_count):  
        self.name = gate_name
        
        self.xdata = xdata.reshape(4, 2)
        self.tdata = tdata.reshape(4, 1)
        
        self.W2 = np.random.rand(i_node, h1_node)             
        self.b2 = np.random.rand(h1_node)
        
        self.W3 = np.random.rand(h1_node, o_node) 
        self.b3 = np.random.rand(o_node)
                        
        self.learning_rate = learning_rate
        self.it_count = it_count
        
        print(self.name + " object is created")

    def feed_forward(self):
        delta = 1e-7   
        
        Z2 = np.dot(self.xdata, self.W2) + self.b2
        A2 = sigmoid(Z2)
        
        Z3 = np.dot(A2, self.W3) + self.b3
        y = sigmoid(Z3)
        
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )

    def loss_val(self):
        delta = 1e-7   
    
        Z2 = np.dot(self.xdata, self.W2) + self.b2    
        A2 = sigmoid(Z2)    
    
        Z3 = np.dot(A2, self.W3) + self.b3    
        y = sigmoid(Z3)    
        
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
                       
    def train(self):
        f = lambda x : self.feed_forward()
        
        print('\n=================================================')
        print('initial W2 =', self.W2)
        print('initial b2 =', self.b2)
        print('initial W3 =', self.W3)
        print('initial b3 =', self.b3)
        print("Initial loss value =", self.loss_val())
        print('=================================================\n')
        
        start_time = datetime.now()
                       
        for step in  range(self.it_count):
            self.W2 -= self.learning_rate * numerical_derivative( f, self.W2 )
            self.b2 -= self.learning_rate * numerical_derivative( f, self.b2 )

            self.W3 -= self.learning_rate * numerical_derivative( f, self.W3 )
            self.b3 -= self.learning_rate * numerical_derivative( f, self.b3 )
    
            if (step % 1000 == 0):
                print("step =", step, "loss value =", self.loss_val())
                
        end_time = datetime.now()
        print('\n=================================================')
        print('updated W2 =', self.W2)
        print('updated b2 =', self.b2)
        print('updated W3 =', self.W3)
        print('updated b3 =', self.b3)
        print("updated loss value =", self.loss_val())
        print("Elapsed Time =>", end_time - start_time)
        print('=================================================\n')
                       
    def predict(self, x):
        z2 = np.dot(x, self.W2) + self.b2
        a2 = sigmoid(z2)
    
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    # 정확도 예측 함수
    def accuracy1(self, test_xdata, test_tdata):
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(True)
            else:
                not_matched_list.append(False)
        
        accuracy_result = len(matched_list) / len(test_xdata)
        print("\nAccuracy =>", accuracy_result)
        
        return matched_list, not_matched_list
    
    def accuracy2(self, test_data):
        matched_list = []
        not_matched_list = []
        
        test_xdata = test_data[:, 0:-1]
        test_tdata = test_data[:, -1]
        
        for index in range(len(test_xdata)):            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(True)
            else:
                not_matched_list.append(False)
        
        accuracy_result = len(matched_list) / len(test_xdata)
        print("\nAccuracy =>", accuracy_result)
        
        return matched_list, not_matched_list

## AND

In [49]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])
i_node = 2
h1_node = 1
o_node = 1
learning_rate = 1e-1
it_count = 10001

AND_obj = LogicGate("AND_GATE", xdata, tdata, i_node, h1_node, o_node, learning_rate, it_count)

AND_obj.train()

AND_GATE object is created

initial W2 = [[0.88599618]
 [0.09249293]]
initial b2 = [0.07582539]
initial W3 = [[0.23693023]]
initial b3 = [0.73368706]
Initial loss value = 4.008109227501867

step = 0 loss value = 3.5989465349656093
step = 1000 loss value = 0.11957388948502107
step = 2000 loss value = 0.03924479386248844
step = 3000 loss value = 0.02296315724178658
step = 4000 loss value = 0.01613388722606274
step = 5000 loss value = 0.012403655332046945
step = 6000 loss value = 0.01006054704028209
step = 7000 loss value = 0.008454954529582594
step = 8000 loss value = 0.007287285551093673
step = 9000 loss value = 0.006400522598411786
step = 10000 loss value = 0.005704542382406904

updated W2 = [[5.48033952]
 [5.48033939]]
updated b2 = [-8.12004468]
updated W3 = [[14.36826415]]
updated b3 = [-7.52341169]
updated loss value = 0.005704542382406904
Elapsed Time => 0:00:02.511290



In [55]:
print(AND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

for data in test_xdata:
    (real_val, logical_val) = AND_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = AND_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = AND_obj.accuracy2(test_data)

print("\n", accuracy_ret)

AND_GATE 

real_val = [0.00054231] , logical_val = 0
real_val = [0.00140529] , logical_val = 0
real_val = [0.00140529] , logical_val = 0
real_val = [0.99765282] , logical_val = 1

Accuracy => 1.0

 ([True, True, True, True], [])

Accuracy => 1.0

 ([True, True, True, True], [])


## OR

In [58]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])
i_node = 2
h1_node = 1
o_node = 1
learning_rate = 1e-1
it_count = 10001

OR_obj = LogicGate("OR_GATE", xdata, tdata, i_node, h1_node, o_node, learning_rate, it_count)

OR_obj.train()

OR_GATE object is created

initial W2 = [[0.91676345]
 [0.36726174]]
initial b2 = [0.3548091]
initial W3 = [[0.72183876]]
initial b3 = [0.61778527]
Initial loss value = 2.156382692545393

step = 0 loss value = 2.154333584468779
step = 1000 loss value = 0.07193441092087949
step = 2000 loss value = 0.03080787209298064
step = 3000 loss value = 0.019419460444791186
step = 4000 loss value = 0.014138479689732183
step = 5000 loss value = 0.011101673544645594
step = 6000 loss value = 0.009132515604553519
step = 7000 loss value = 0.007753440530923274
step = 8000 loss value = 0.00673436474016906
step = 9000 loss value = 0.005950904662171268
step = 10000 loss value = 0.005329985409227789

updated W2 = [[6.4696936 ]
 [6.46920369]]
updated b2 = [-3.44154654]
updated W3 = [[13.88779489]]
updated b3 = [-6.18541105]
updated loss value = 0.005329985409227789
Elapsed Time => 0:00:02.494362



In [59]:
print(OR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

for data in test_xdata:
    (real_val, logical_val) = OR_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = OR_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = OR_obj.accuracy2(test_data)

print("\n", accuracy_ret)

OR_GATE 

real_val = [0.00315822] , logical_val = 0
real_val = [0.99914271] , logical_val = 1
real_val = [0.99914297] , logical_val = 1
real_val = [0.99954798] , logical_val = 1

Accuracy => 1.0

 ([True, True, True, True], [])

Accuracy => 1.0

 ([True, True, True, True], [])


## NAND

In [60]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])
i_node = 2
h1_node = 1
o_node = 1
learning_rate = 1e-1
it_count = 10001

NAND_obj = LogicGate("NAND_GATE", xdata, tdata, i_node, h1_node, o_node, learning_rate, it_count)

NAND_obj.train()

NAND_GATE object is created

initial W2 = [[0.92644788]
 [0.24892935]]
initial b2 = [0.60217909]
initial W3 = [[0.12098546]]
initial b3 = [0.03606468]
Initial loss value = 2.665254678248839

step = 0 loss value = 2.569181049787548
step = 1000 loss value = 0.12754894009156237
step = 2000 loss value = 0.04012524092969623
step = 3000 loss value = 0.02326052941468509
step = 4000 loss value = 0.016277987121284518
step = 5000 loss value = 0.012487163647265001
step = 6000 loss value = 0.010114400969624428
step = 7000 loss value = 0.008492242110763217
step = 8000 loss value = 0.007314443891157566
step = 9000 loss value = 0.006421067459088539
step = 10000 loss value = 0.005720548336445282

updated W2 = [[5.47239083]
 [5.47239072]]
updated b2 = [-8.10826714]
updated W3 = [[-14.36863255]]
updated b3 = [7.52359651]
updated loss value = 0.005720548336445282
Elapsed Time => 0:00:02.520265



In [61]:
print(NAND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

for data in test_xdata:
    (real_val, logical_val) = NAND_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = NAND_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = NAND_obj.accuracy2(test_data)

print("\n", accuracy_ret)

NAND_GATE 

real_val = [0.99945777] , logical_val = 1
real_val = [0.99859012] , logical_val = 1
real_val = [0.99859012] , logical_val = 1
real_val = [0.00235404] , logical_val = 0

Accuracy => 1.0

 ([True, True, True, True], [])

Accuracy => 1.0

 ([True, True, True, True], [])


## XOR

In [62]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])
i_node = 2
h1_node = 2
o_node = 1
learning_rate = 1e-1
it_count = 10001

XOR_obj = LogicGate("XOR_GATE", xdata, tdata, i_node, h1_node, o_node, learning_rate, it_count)

XOR_obj.train()

XOR_GATE object is created

initial W2 = [[0.23511572 0.98219719]
 [0.06961408 0.36464519]]
initial b2 = [0.24138343 0.71276864]
initial W3 = [[0.75812504]
 [0.87590535]]
initial b3 = [0.42385481]
Initial loss value = 3.8784975755898548

step = 0 loss value = 3.56443938121291
step = 1000 loss value = 1.5502654572805483
step = 2000 loss value = 0.11578116936525933
step = 3000 loss value = 0.05370016549078655
step = 4000 loss value = 0.03460032551752469
step = 5000 loss value = 0.025431557745503784
step = 6000 loss value = 0.020069673090899208
step = 7000 loss value = 0.016558927451978064
step = 8000 loss value = 0.01408493393876472
step = 9000 loss value = 0.012249025952551267
step = 10000 loss value = 0.01083333129232917

updated W2 = [[5.66880075 7.46686397]
 [5.66818595 7.46359566]]
updated b2 = [-8.6682477  -3.41925369]
updated W3 = [[-13.75990388]
 [ 12.97562331]]
updated b3 = [-6.0839614]
updated loss value = 0.01083333129232917
Elapsed Time => 0:00:04.557820



In [63]:
print(XOR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

for data in test_xdata:
    (real_val, logical_val) = XOR_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = XOR_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = XOR_obj.accuracy2(test_data)

print("\n", accuracy_ret)

XOR_GATE 

real_val = [0.0034189] , logical_val = 0
real_val = [0.99756578] , logical_val = 1
real_val = [0.99756659] , logical_val = 1
real_val = [0.0025322] , logical_val = 0

Accuracy => 1.0

 ([True, True, True, True], [])

Accuracy => 1.0

 ([True, True, True, True], [])


ver1 의 경우 클래스 내에서 data를 분리하기 때문에 나중에 문제가 될 가능성이 많다.  
ver2 에서는 데이터를 객체 생성 시 넣지 않는 이유.